# River Pointcloud
### Assign depths to corresponding X and Y coordinates of the river pointcloud by making bins of the RTK track polyline Y coordinate

Please note: the RTK waterline track and the left/right bank tracks are included, however they are separated afterwards in order to get a better rasterization result.  
Sten Schurer

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import re

#### Import data

In [2]:
river = pd.DataFrame(pd.read_csv(os.path.join(r'C:\Users\SSchurer\Documents\TU_Delft\Thesis\TNDTT\River', 'XYD.csv')))
rtk_poly = pd.DataFrame(pd.read_csv(os.path.join(r'C:\Users\SSchurer\Documents\TU_Delft\Thesis\TNDTT\River', 'rtk_poly_extrapolated.txt'), delimiter = ' '))
rtk_track = pd.read_excel(os.path.join(r'C:\Users\SSchurer\Documents\TU_Delft\Thesis\RTK\17-12', 'waterline_height_XYZ.xls'))
rtk_poly.columns = ['Y', 'Z']
rtk_track.columns = ['X', 'Y', 'Z']
print(river)
print(rtk_poly)
print(rtk_track)

                  X             Y  Depth
0     200613.196892  8.340895e+06  1.238
1     200613.181050  8.340894e+06  1.245
2     200613.198137  8.340894e+06  1.238
3     200613.240594  8.340894e+06  1.252
4     200613.287502  8.340894e+06  1.238
...             ...           ...    ...
4262  200661.853407  8.340367e+06  0.493
4263  200661.755314  8.340366e+06  0.555
4264  200661.617030  8.340365e+06  0.380
4265  200661.457264  8.340364e+06  0.414
4266  200661.261217  8.340363e+06  0.290

[4267 rows x 3 columns]
                 Y           Z
0     8.341003e+06  352.220007
1     8.341003e+06  352.219761
2     8.341002e+06  352.219516
3     8.341002e+06  352.219271
4     8.341001e+06  352.219025
...            ...         ...
1494  8.340222e+06  351.853539
1495  8.340221e+06  351.853293
1496  8.340221e+06  351.853048
1497  8.340220e+06  351.852803
1498  8.340220e+06  351.852557

[1499 rows x 2 columns]
                 X             Y        Z
0    200498.394202  8.340838e+06  352.140
1 

#### Make bins of the Y coordinate polynomial of the RTK waterline track
#### Subdivide the river point cloud X,Y coordinates in the bins

In [3]:
indices = np.digitize(x = river['Y'], bins = rtk_poly['Y'], right = False)
print(indices.min())
print(len(indices))

208
4267


In [4]:
height = rtk_poly.iloc[indices]
print(height)

                 Y           Z
208   8.340894e+06  352.168986
208   8.340894e+06  352.168986
208   8.340894e+06  352.168986
208   8.340894e+06  352.168986
208   8.340894e+06  352.168986
...            ...         ...
1217  8.340367e+06  351.921485
1219  8.340365e+06  351.920994
1220  8.340365e+06  351.920749
1222  8.340364e+06  351.920258
1224  8.340363e+06  351.919768

[4267 rows x 2 columns]


#### Subtract the depth from the bin heights

In [5]:
height.index = np.arange(height.shape[0])
river['Z'] = height['Z'] - river['Depth']
xyz_river = river.drop('Depth', axis=1)
River_XYZ = pd.concat([xyz_river, rtk_track])
print(River_XYZ)

                 X             Y           Z
0    200613.196892  8.340895e+06  350.930986
1    200613.181050  8.340894e+06  350.923986
2    200613.198137  8.340894e+06  350.930986
3    200613.240594  8.340894e+06  350.916986
4    200613.287502  8.340894e+06  350.930986
..             ...           ...         ...
892  200653.774242  8.340428e+06  352.081000
893  200653.777008  8.340428e+06  352.098000
894  200653.779865  8.340428e+06  352.083000
895  200653.785084  8.340428e+06  352.060000
896  200653.781671  8.340428e+06  352.084000

[5164 rows x 3 columns]


In [6]:
# np.savetxt(r'C:\Users\SSchurer\Documents\TU_Delft\Thesis\TNDTT\River\River_XYZ.txt', River_XYZ)